TensorFlow Hub is a way to share pretrained model components
This tutorial demonstrates:
1. Hot to use TensorFlow Hub with tf.kears
2. How to do image classification using TensorFlow Hub.
3. How to do simple transfer learning.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt

try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

In [0]:
!pip install -q -U tensorflow_hub
import tensorflow_hub as hub

from tensorflow.keras import layers

## Download the classifier
Use `hub.module` to load a mobilenet, and `tf.keras.layers.Lambda` to wrap it up as a keras layer.

In [0]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [0]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3, ))
])

## Run it on a single image
Download a single image to try the model on.

In [0]:
import numpy as np
import PIL.Image as Image

grace_hopper = tf.keras.utils.get_file('image.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [0]:
grace_hopper = np.array(grace_hopper) / 255.0
grace_hopper.shape

In [0]:
grace_hopper[np.newaxis, ...].shape

In [0]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

The result is a 1001 element vector of logits, rating the probability of each class for the image
Unnormalized log probability

In [0]:
# Top class ID can be found with argmax:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

## Decode the preditions
We have the predicted class ID, Fetch the `ImageNet` labels, and decode the predictions

In [0]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
# string data split '\n' to array.

In [0]:
imagenet_labels

In [0]:
plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title('Prediction: ' + predicted_class_name.title())

## Simple transfer learning
Using TF Hub it is simple to retrain the top layer of the model to recognize the classes in our dataset.

In [0]:
# Dataset
data_root = tf.keras.utils.get_file(
    'flower_photos', 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True
)

In [0]:
# All of TensorFlow Hub's image modules expect float inputs in the [0, 1] range.
# Use the `ImageDataGenerator`'s rescale parameter to achieve this.

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root), target_size=IMAGE_SHAPE,
                                                 batch_size=32)

In [0]:
# The resulting object is an iterator that returns image_batch, label_batch pairs.
for image_batch, label_batch in image_data:
    print("image batch shape: ", image_batch.shape);
    print("label batch shape: ", label_batch.shape);
    break

In [0]:
# Run the classifier on a batch of images
result_batch = classifier.predict(image_batch)
result_batch.shape

In [0]:
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

In [0]:
plt.figure(figsize=(10, 9))
plt.subplots_adjust(hspace=0.5)

for i in range(30):
    plt.subplot(6, 5, i + 1)
    plt.imshow(image_batch[i])
    plt.title(predicted_class_names[i])
    plt.axis('off')
_ = plt.suptitle('ImageNet predictions')

## Download the headless model
TensorFlow Hub aslo distributes models without the top classification layer. These can be used to easily do transfer learning

In [0]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}

In [0]:
# Create the feature extractor
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224, 224, 3))

In [0]:
# It returns a 1280-length vector for each image
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

In [0]:
# Freeze the variables in the feature extractor layer, so that
# the training only modifies the new classifier layer
feature_extractor_layer.trainable = False

In [0]:
# Attach a classification head
# Now wrap the hub layer in a `tf.keras.Sequential` model, and add a new
# classification layer.
model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(image_data.num_classes, activation='softmax')
])

In [0]:
model.summary()

In [0]:
predictions = model.predict(image_batch)

In [0]:
predictions.shape

In [0]:
# Train the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['acc']
)

In [0]:
# To visualize training process, use a custom callback
# to log the loss and accuracy of each batch individualy,
# instead of the epoch average.
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_acc = []

    def on_train_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['acc'])
        self.model.reset_metrics()

In [0]:
steps_per_epoch = np.ceil(image_data.samples / image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit_generator(image_data,
                              epochs=15,
                              steps_per_epoch=steps_per_epoch,
                              callbacks=[batch_stats_callback])

In [0]:
plt.figure()
plt.ylabel('loss')
plt.xlabel('Training Steps')
plt.ylim([0, 2])
plt.plot(batch_stats_callback.batch_losses)

In [0]:
plt.figure()
plt.ylabel('Accuracy')
plt.xlabel('Training Steps')
plt.ylim([0, 1])
plt.plot(batch_stats_callback.batch_acc)

In [0]:
# Check the predictions
# To redo the plot from before, first get the ordered list of class names:
class_names = sorted(image_data.class_indices.items(), key=lambda pair: pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

In [0]:
# Run the image batch through the model
# and convert the indices to class names
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

In [0]:
# Plot the result
label_id = np.argmax(label_batch, axis=-1)

In [0]:
plt.figure(figsize=(10, 9))
plt.subplots_adjust(hspace=0.5)
for i in range(30):
    plt.subplot(6, 5, i+1)
    plt.imshow(image_batch[i])
    color = 'green' if predicted_id[i] == label_id[i] else 'red'
    plt.title(predicted_label_batch[i], color=color)
    plt.axis('off')
_ = plt.suptitle('Model predictions (green: correct, red: incorrect')